In [1]:
def reviews ( local ):

    url = "https://www.google.com/maps"
    
    # Import the necessary libraries.
    import os
    import time
    import pandas as pd
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium import webdriver
    from selenium.webdriver.support.ui import Select
    from selenium import webdriver
    from bs4 import BeautifulSoup
    from unidecode import unidecode

    # Store the function's input as "name".
    name = local

    # Run the driver.
    driver = webdriver.Chrome()
    driver.get(url)

    # Run through cookies page on Google.
    button_xpath = "//button[@jsname='b3VHJd']"
    button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
    button.click()

    # Write the local's name in the search bar and run it.
    searchbar = driver.find_element(By.ID, "searchboxinput")
    searchbar.send_keys(name)
    searchbar.send_keys(Keys.RETURN)

    # Open the reviews section from that local's brochure.
    button_xpath = "//button[@jslog='145620; track:click;']"
    button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
    button.click()

    # Execute BeautifulSoup to read what is in that same page.
    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")
    time.sleep(3)

    # Set the number of reviews to scrape, 500 was a reasonable number as a higher one gave problems.
    reviews_number = 100

    # We scrape the number of reviews available in case they have less than 500, eitherwise we would be going into an infinte while loop later on.
    reviews_available = int(soup.find_all("div",{"class":"fontBodySmall"})[3].text.split(" ")[0].replace(",", ""))
    if reviews_available > reviews_number:
        reviews_to_scrape = reviews_number
    else:
        reviews_to_scrape = reviews_available

    # Open the sort button.
    button_xpath = "g88MCb"
    button =  WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, button_xpath)))
    button[2].click()

    # Click the "newest" drop-down button to sort by date, ascending (newest first). We will be extractnig the last 500 reviews.
    button_xpath = "fxNQSd"
    newest = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, button_xpath)))
    newest[1].click()

    # Run the scrolling while loop until the number of reviews on the available scrolled page is higher than the set number of 500.
    while len(soup.find_all('span', {"class":"kvMYJc"})) < reviews_to_scrape:
        scroll = driver.find_element(By.XPATH, "/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]")
        scroll.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        html = driver.execute_script("return document.body.outerHTML;")
        soup = BeautifulSoup(html, "html.parser")
        
        print('\r' + str(round((((len(soup.find_all('span', {"class":"kvMYJc"}))) / reviews_to_scrape) * 100), 2))+"%", end='', flush=True)

    time.sleep(1)

    # Once we have 500 reviews on our visible page, run through all of them and click the "more" button to access the full reviews.
    button_xpath = "w8nwRe"
    button = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, button_xpath)))

    for i in button:
        i.click()

    time.sleep(3)

    # BeautifulSoup to read all 500 reviews.
    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    # Create a ratings list with all the 500 ratings (from 1 to 5).
    rating = soup.find_all('span', {"class":"kvMYJc"})
    ratings = []

    for i in rating:
        ratings.append(i["aria-label"][0])

    # Create lists for the reviews, usernames, how long ago where the reviews made, and the link to that review to be able to answer the user's review.
    reviews = []
    usernames = []
    times = []
    linktoreviews = []

    for i in range(len(ratings)):
        try:
            newratings = soup.find_all('div', {"class":"jftiEf fontBodyMedium"})[i].find('span', class_='wiI7pd').text
            reviews.append(unidecode(newratings))

        except AttributeError:
            reviews.append("No Review")
        
        username = soup.find_all('div', {"class":"jftiEf fontBodyMedium"})[i]["aria-label"]
        usernames.append(unidecode(username))

        time = soup.find_all('div', {"class":"jftiEf fontBodyMedium"})[i].find('span', class_='rsqaWe').text
        times.append(time)

        linktoreview = soup.find_all('div', {"class":"jftiEf fontBodyMedium"})[i].find('button', class_='al6Kxe')["data-href"]
        linktoreviews.append(linktoreview)
    
    # System to transform the time data into days (3 months ago = 90, 2 weeks ago = 14, and so on).
    times2 = []
    days = []

    for i in times:
        times2.append(i.split(" ")[:2])

    index = {"minute": 1440, "hour": 24, "day": 1, "week": 7, "month": 30, "year": 365}

    for j in times2:
        if j[0] == ("a" or "an"):
            for i in index.keys():
                if i in j[1]:
                    if i == "hour":
                        days.append(1 / index[i])
                    elif i == "minute":
                        days.append(1 / index[i])
                    else:
                        days.append(1 * index[i])
        else: 
            for i in index.keys():
                if i in j[1]:
                    if i == "hour":
                        days.append(int(j[0]) / index[i])
                    elif i == "minute":
                        days.append(int(j[0]) / index[i])
                    else:
                        days.append(int(j[0]) * index[i])

    # Create the dictionary where we will store all the extracted data.  
    table = {}
    table["restaurant"] = name
    table["username"] = usernames
    table["review"] = reviews
    table["rating"] = ratings
    table["daysago"] = days
    table["linktoreview"] = linktoreviews

    # Convert dictionary to pandas dataframe.
    restaurant = pd.DataFrame(table)

    # Export as csv file with the name of the file to be the inputted local's name.
    restaurant.to_csv(f"{name}.csv")

In [2]:
def information ( local ):
    import os
    import time
    import pandas as pd

    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium import webdriver
    from selenium.webdriver.support.ui import Select
    from selenium import webdriver

    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    from bs4 import BeautifulSoup
    from unidecode import unidecode
    import nltk

    url = "https://www.google.com/maps"

    name = local

    driver = webdriver.Chrome()
    driver.get(url)

    button_xpath = "//button[@jsname='b3VHJd']"
    button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
    button.click()

    searchbar = driver.find_element(By.ID, "searchboxinput")
    searchbar.send_keys(name)
    searchbar.send_keys(Keys.RETURN)

    time.sleep(5)

    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    company = []
    company.append(soup.find_all("h1", {"class":"DUwDvf lfPIob"})[0].text)

    description = []
    description.append(soup.find_all("button", {"class":"DkEaL"})[0].text)

    address = []
    address.append(soup.find_all("div", {"class":"Io6YTe fontBodyMedium kR99db"})[0].text)

    information = {}
    information["company"] = company
    information["description"] = description
    information["address"] = address

    informations = pd.DataFrame(information)
    # informations.to_csv("locals.csv")

    locals = pd.read_csv("locals.csv")
    locals = locals.iloc[:, 1:]

    table = pd.concat([locals, informations])

    # table = table.iloc[:, 1:]
    table = table.drop_duplicates().reset_index()
    table.drop(columns=['index'], inplace=True)
    table.to_csv("locals.csv")

In [3]:
local = input()
reviews(local)
information(local)

<selenium.webdriver.remote.webelement.WebElement (session="0155e7335c0ec52f503f7312312f69f0", element="745F47A6556A502864F2678A62219337_element_126")>
10.0%<selenium.webdriver.remote.webelement.WebElement (session="0155e7335c0ec52f503f7312312f69f0", element="745F47A6556A502864F2678A62219337_element_126")>
10.0%<selenium.webdriver.remote.webelement.WebElement (session="0155e7335c0ec52f503f7312312f69f0", element="745F47A6556A502864F2678A62219337_element_126")>


KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.read_csv("locals.csv", index_col=0)

In [ ]:
df = pd.read_csv("locals.csv", index_col=0)
df